### 02 – Agents and Short-Term Memory

In the previous module, we manually orchestrated tools: check if model wants a tool, execute it, send back result, repeat.

**Agents automate this loop.** They decide:
- Which tool to use (if any)
- How many times to iterate
- When they have enough information to answer

This module covers:
1. What agents are and why they matter
2. Creating agents with LangChain v1 APIs
3. Adding short-term memory for context across turns
4. Structured outputs for reliable parsing
5. Debugging agent behavior

By the end, you'll understand the foundation for building complex, stateful AI applications.

In [1]:
# Core LangChain
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_groq import ChatGroq

# Agent creation
from langchain.agents import create_agent

# Memory and state management
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# For structured output
from pydantic import BaseModel, Field
from typing import List, Optional

# Utilities
import json
from datetime import datetime

/Users/yashsarode/Downloads/Personal Projects/Python/LangGraph-personal/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
# Initialize model
from typing import Annotated, Literal


llm = ChatGroq(
    model="meta-llama/llama-4-maverick-17b-128e-instruct",
    temperature=0,
)

@tool
def calculator(
    operation: Annotated[Literal["add", "subtract", "multiply", "divide"], "The math operation"],
    a: Annotated[float, "First number"],
    b: Annotated[float, "Second number"]
) -> str:
    """Perform basic arithmetic operations. Use for exact calculations."""
    ops = {
        "add": a + b,
        "subtract": a - b,
        "multiply": a * b,
        "divide": a / b if b != 0 else "Error: Division by zero"
    }
    result = ops.get(operation.lower(), "Invalid operation")
    return f"Result: {result}"

@tool
def get_current_time(timezone: str = "local") -> str:
    """Get the current date and time."""
    current = datetime.now()
    return f"Current time: {current.strftime('%Y-%m-%d %H:%M:%S')}"

@tool
def search_database(query: Literal["laptop", "phone", "tablet"]) -> str:
    """Search a simulated product database. Use when user asks about products or inventory."""
    # Simulated product database
    products = {
        "laptop": "In stock: 15 units, Price: $999",
        "phone": "In stock: 42 units, Price: $699",
        "tablet": "Out of stock, Expected: Next week",
    }
    result = products.get(query.lower(), f"No product found matching '{query}'")
    return result

tools = [calculator, get_current_time, search_database]

print(f"Model and {len(tools)} tools ready")

Model and 3 tools ready


### What is an Agent?

**Manual tool calling** (previous module):
- We wrote the orchestration loop ourselves
- We decided when to call tools and when to stop

**Agent**:
- Autonomous decision-maker
- Decides which tool to call based on the query
- Can call multiple tools in sequence
- Knows when it has enough information to answer

**The agent loop:**
```
Query → Agent → Call Tool? → Execute → Agent → Call Another Tool? → Final Answer
```

Agents use ReAct pattern (Reasoning + Acting): think about what to do, then do it, then think again.

### When to Use Agents vs Simple Chains

**Use simple tool calling when:**
- You know exactly which tool(s) to call
- Single-step operations
- Deterministic workflows

**Use agents when:**
- The tool choice depends on user input
- Multiple tools might be needed
- You need multi-step reasoning
- Queries are open-ended

Example: "What's 25% of the laptop price?" needs both search_database and calculator.

In [26]:

system_prompt = """You are a helpful assistant with access to tools.

When a user asks a question:
1. Think about which tool(s) you need
2. Call the appropriate tools
3. Provide a clear, concise answer

Be direct and avoid unnecessary explanations of your process."""

# Create the agent
# In LangChain v1, create_agent is the standard way to build agents
agent = create_agent(
    llm,
    tools,
    system_prompt=system_prompt,
)

print("Agent created")
print(f"Agent type: {type(agent)}")

Agent created
Agent type: <class 'langgraph.graph.state.CompiledStateGraph'>


### Agent as a Runnable

Agents implement the Runnable interface, which means:
- `.invoke()` for single execution
- `.batch()` for multiple queries
- `.stream()` for streaming responses
- Can be composed into larger pipelines

This consistency is what makes LangChain powerful.

In [16]:
# Single query - agent has no memory of previous interactions
query = "What is current date and time?"

response = agent.invoke({"messages": [HumanMessage(content=query)]})

print(f"Query: {query}")
print(f"\nAgent response:")
print(response["messages"][-1].content)

[print(msg.pretty_print()) for msg in response["messages"]]

Query: What is current date and time?

Agent response:
The current date and time is December 1, 2025, 21:57:02.
================================ Human Message =================================

What is current date and time?
None
================================== Ai Message ==================================
Tool Calls:
  get_current_time (fdm66vhh7)
 Call ID: fdm66vhh7
  Args:
    timezone: local
None
================================= Tool Message =================================
Name: get_current_time

Current time: 2025-12-01 21:57:02
None
================================== Ai Message ==================================

The current date and time is December 1, 2025, 21:57:02.
None


[None, None, None, None]

In [29]:
# Query that requires multiple tools
query = "What is the laptop price in the database? After getting the price, calculate 15% of the price."

response = agent.invoke({"messages": [HumanMessage(content=query)]})

print(f"Query: {query}")
print(f"\nAgent response:")
print(response["messages"][-1].content)
[print(msg.pretty_print()) for msg in response["messages"]]

Query: What is the laptop price in the database? After getting the price, calculate 15% of the price.

Agent response:
The laptop price is $999, and 15% of the price is $149.85.
================================ Human Message =================================

What is the laptop price in the database? After getting the price, calculate 15% of the price.
None
================================== Ai Message ==================================
Tool Calls:
  search_database (0m6d6c59h)
 Call ID: 0m6d6c59h
  Args:
    query: laptop
None
================================= Tool Message =================================
Name: search_database

In stock: 15 units, Price: $999
None
================================== Ai Message ==================================
Tool Calls:
  calculator (6tt0ggvem)
 Call ID: 6tt0ggvem
  Args:
    a: 999
    b: 0.15
    operation: multiply
None
================================= Tool Message =================================
Name: calculator

Result: 149.85
None
========

[None, None, None, None, None, None]

### Short-Term Memory in Agents

Until now, our agent forgets everything after each call. Each invocation is independent.

**The problem:**
- User: "Search for laptop"
- Agent: "In stock: 15 units, Price: $999"
- User: "What's 20% of that price?"
- Agent: "What price are you referring to?"

**The solution:** Store conversation history and pass it with each request.

In LangChain v1.x, we use the `state_schema` parameter in `create_agent` to define what state to track.

In [ ]:
from typing import TypedDict
from langchain_core.messages import BaseMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import MessagesState

# Create a checkpointer for in-memory storage
# This stores conversation state between agent calls
memory = MemorySaver()

# Create agent with memory using state_schema
agent_with_memory = create_agent(
    llm,
    tools,
    system_prompt=system_prompt,
    state_schema=MessagesState,  # Built-in state that tracks messages
    checkpointer=memory,          # Persists state between calls
)

print("Agent with memory created")
print("State schema: MessagesState (tracks conversation history)")

Agent with memory created
State schema: MessagesState (tracks conversation history)


### How Memory Works in LangChain v1.x

**Key concepts:**

1. **state_schema**: Defines what data to track (messages, custom fields, etc.)
2. **checkpointer**: Where to store state (memory, Redis, Postgres, etc.)
3. **thread_id**: Session identifier - like a conversation ID

**Flow:**
```
User message → Agent (loads history from thread_id) → Tool calls → Response → Save to thread_id
```

Each thread_id maintains its own isolated conversation history.

In [33]:
def chat_with_memory(message: str, thread_id: str):
    """
    Chat with memory-enabled agent.
    thread_id acts as session identifier.
    """
    
    # Configuration with thread_id for memory lookup
    config = {"configurable": {"thread_id": thread_id}}
    
    # Invoke agent - it automatically loads and saves history
    response = agent_with_memory.invoke(
        {"messages": [HumanMessage(content=message)]},
        config=config
    )
    
    # Return the last message (agent's response)
    return response["messages"][-1].content

# Start a conversation
thread = "conversation_1"

print("Turn 1:")
response1 = chat_with_memory("Search for laptop", thread_id=thread)
print(response1)

print("\n" + "="*60)
print("Turn 2 (agent remembers context):")
response2 = chat_with_memory("What's 20% of that price?", thread_id=thread)
print(response2)

print("\n" + "="*60)
print("Turn 3:")
response3 = chat_with_memory("How many are available?", thread_id=thread)
print(response3)

Turn 1:
There are 15 laptops in stock, priced at $999.

Turn 2 (agent remembers context):
20% of $999 is $199.80.

Turn 3:
There are 15 laptops available.


In [40]:
# Different threads maintain separate histories

print("THREAD A - Alice:")
print("-"*60)
chat_with_memory("I want to buy a phone", thread_id="alice")
response_a = chat_with_memory("How much does it cost?", thread_id="alice")
print(response_a)

print("\n" + "="*60)
print("THREAD B - Bob:")
print("-"*60)
chat_with_memory("I want to buy a tablet", thread_id="bob")
response_b = chat_with_memory("When will it be available?", thread_id="bob")
print(response_b)

print("\n" + "="*60)
print("Back to THREAD A - Alice:")
print("-"*60)
# Alice's context is preserved
response_a2 = chat_with_memory("Can you calculate 15% discount?", thread_id="alice")
print(response_a2)

THREAD A - Alice:
------------------------------------------------------------
The phone costs $699.

THREAD B - Bob:
------------------------------------------------------------
The tablet is expected to be available next week.

Back to THREAD A - Alice:
------------------------------------------------------------
The 15% discount on the phone is $104.85, making the new price $594.15.


### Structured Output from Agents

Sometimes you need agents to return data in a specific format, not just text.

**Use cases:**
- Extract structured information (names, dates, amounts)
- Form filling from conversation
- API integration requiring specific JSON
- Database operations

We define a Pydantic schema and the agent returns validated data.

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional

class ProductInquiry(BaseModel):
    """Structured information extracted from product queries"""
    product_name: str = Field(description="Name of the product user asked about")
    price: Optional[float] = Field(default=None, description="Product price if mentioned")
    stock_quantity: Optional[int] = Field(default=None, description="Available quantity if mentioned")
    user_wants_calculation: bool = Field(description="Whether user requested a calculation")
    calculation_result: Optional[float] = Field(default=None, description="Result of calculation if performed")

# Agent that returns structured data
structured_agent = create_agent(
    llm,
    tools,
    system_prompt="""You are an assistant that extracts structured information.

After gathering information via tools, return your findings in the specified structured format.
Be precise with the data extraction.""",
    state_schema=MessagesState,
    response_format=ProductInquiry,  # Enforce this output structure
)
# response_format = ToolStrategy(ProductInquiry) is the way to do in LangChain v1 as per docs, but some open source models don't support it
print("Structured output agent created")

Structured output agent created


In [49]:
# Query that should produce structured output
query = "What's the laptop price and how many do we have in stock?"

try:
    response = structured_agent.invoke(
        {"messages": [HumanMessage(content=query)]},
        config={"configurable": {"thread_id": "structured_test"}}
    )
    
    # The agent's response will be structured according to ProductInquiry
    final_message = response["messages"][-1]
    
    print("Structured output:")
    print("="*60)
    
    # If the model supports structured output, content will be JSON
    if isinstance(final_message.content, str):
        try:
            parsed = json.loads(final_message.content)
            print(json.dumps(parsed, indent=2))
        except:
            print(final_message.content)
    else:
        print(final_message.content)
        
except Exception as e:
    print(f"Note: Structured output requires model support")
    print(f"Error: {e}")
    print("\nYour model may not support response_format parameter")
    print("This is fine - structured output is an advanced feature")

Structured output:
Returning structured response: product_name='laptop' price=999.0 stock_quantity=15 user_wants_calculation=False calculation_result=None


### Understanding response_format

**How it works:**
1. You provide a Pydantic model as `response_format`
2. LangChain converts it to a JSON schema
3. The model is instructed to return data matching that schema
4. Output is automatically validated against your model

**Requirements:**
- Model must support structured output (OpenAI, Anthropic Claude, some others)
- Not all models support this feature
- Groq models may have limited support depending on the specific model

**Alternative approach:** Parse agent text output yourself using another LLM call or traditional parsing.

### State Schema: Beyond Messages

So far we've used `MessagesState` - it only tracks conversation history.

But you can define custom state to track additional information:
```python
from typing import TypedDict

class CustomState(TypedDict):
    messages: List[BaseMessage]  # Required
    user_preferences: dict        # Custom field
    current_cart: List[str]       # Custom field
    total_cost: float             # Custom field
```

**Why custom state:**
- Track domain-specific data
- Maintain application state alongside conversation
- Enable complex workflows

In the LangGraph module where custom state truly shines.

### Error Handling and Interrupts in Production

LangChain v1.x provides several mechanisms for robust agent execution:

**Tool-level error handling:**
- `@wrap_tool_call` decorator: Customize how individual tool errors are handled
- Prevents one failing tool from crashing the entire agent

**Agent-level interrupts:**
- `interrupt_before`: Pause execution before specific nodes (e.g., for user confirmation)
- `interrupt_after`: Pause after nodes to validate output or add processing

**Use cases:**
- Retry failed tool calls with different strategies
- Request user confirmation before critical actions
- Validate outputs before proceeding
- Graceful degradation when tools fail

The agent will return a ToolMessage with the custom error message when a tool fails:
```python
[
    ...
    ToolMessage(
        content="Tool error: Please check your input and try again. (division by zero)",
        tool_call_id="..."
    ),
    ...
]
```

In [51]:
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage


@wrap_tool_call
def handle_tool_errors(request, handler):
    """Handle tool execution errors with custom messages."""
    try:
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

agent = create_agent(
   llm,
    tools,
    middleware=[handle_tool_errors]
)

agent.invoke({
    "messages": [HumanMessage(content="What will be the total cost of entire laptop stock?")]
})

{'messages': [HumanMessage(content='What will be the total cost of entire laptop stock?', additional_kwargs={}, response_metadata={}, id='786fc002-6037-4372-b529-57c2fe604bbf'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'a78vten8e', 'function': {'arguments': '{"query":"laptop"}', 'name': 'search_database'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 848, 'total_tokens': 878, 'completion_time': 0.024905078, 'completion_tokens_details': None, 'prompt_time': 0.412374327, 'prompt_tokens_details': None, 'queue_time': 0.348187761, 'total_time': 0.437279405}, 'model_name': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': 'fp_9b0c2006ef', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--e1224185-54cb-481a-a099-972d8c3da219-0', tool_calls=[{'name': 'search_database', 'args': {'query': 'laptop'}, 'id': 'a78vten8e', 'type': 'tool